In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

### PRE-PROCESAR LAS IMAGENES

In [ ]:
# Transformaciones para normalizar e igualar tamaño
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Si las imágenes son en escala de grises
    transforms.Resize((48, 48)),                  # Tamaño común en FER-2013
    transforms.RandomRotation(20),                # Rotación aleatoria para aumentar la variabilidad
    transforms.ColorJitter(brightness=(0.8, 1.2)), # Ajuste de brillo
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [ ]:
data_dir = '../Tarea 1/dataset'

### Cargar los datasets

In [37]:
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


#### Muestra de clases

In [38]:
# Clases
classes = train_dataset.classes
print(f"Clases: {classes}")

Clases: ['angry', 'disgust', 'happy', 'sad', 'surprise']


### Inicializar el modelo, la función de pérdida y el optimizador

In [39]:
class EmotionCNN(nn.Module):
    def __init__(self, num_classes=5):
        super(EmotionCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.conv(x)
        x = self.fc(x)
        return x



### Entrenamiento del modelo

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = EmotionCNN(num_classes=len(train_dataset.classes)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenar por 15 épocas
for epoch in range(15):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Época {epoch+1}, pérdida: {running_loss/len(train_loader):.4f}")

Época 1, pérdida: 1.2426
Época 2, pérdida: 0.9520
Época 3, pérdida: 0.8262
Época 4, pérdida: 0.7387
Época 5, pérdida: 0.6624
Época 6, pérdida: 0.5994
Época 7, pérdida: 0.5343
Época 8, pérdida: 0.4764
Época 9, pérdida: 0.4084
Época 10, pérdida: 0.3586
Época 11, pérdida: 0.3092
Época 12, pérdida: 0.2761
Época 13, pérdida: 0.2305
Época 14, pérdida: 0.2060
Época 15, pérdida: 0.1861


### Guardar el modelo

In [42]:
torch.save(model.state_dict(), "modelo_emociones.pth")

### CARGAR MODELO (Por si ya hay uno guardado)

In [ ]:
# Cargar modelo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_classes = 5  # Asegúrate de que sea correcto
model = EmotionCNN(num_classes=num_classes)
model.load_state_dict(torch.load("modelo_emociones.pth", map_location=torch.device("cpu")))  # Usa "cuda" si tienes GPU
model.eval()
model.to(device)

### Ver precision del modelo

In [41]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Precisión en el conjunto de prueba: {100 * correct / total:.2f}%')

Precisión en el conjunto de prueba: 72.99%


### Probar modelo con imagenes

In [46]:
import cv2
import torch
from torchvision import transforms
from PIL import Image

# Asegúrate de tener el modelo cargado y en modo evaluación
model.eval()
model.to(device)

# Transformación igual que en entrenamiento
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Clases
classes = train_dataset.classes

# Ruta de la imagen
img_path = "prueba.jpg"

# Cargar imagen y convertir a escala de grises
img = cv2.imread(img_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# Opcional: detectar rostro y recortar
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(gray, 1.3, 5)

if len(faces) == 0:
    print("No se detectaron rostros.")
else:
    for (x, y, w, h) in faces:
        roi = gray[y:y+h, x:x+w]
        roi = cv2.resize(roi, (48, 48))
        roi = Image.fromarray(roi)

        # Preprocesar y pasar al modelo
        input_tensor = transform(roi).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            emotion = classes[predicted.item()]
            print(f"Emoción detectada: {emotion}")


Emoción detectada: happy


### Probar modelo con camara

In [24]:
import cv2
import torch
import numpy as np
from torchvision import transforms
from PIL import Image

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.eval()
model.to(device)

transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

classes = ['angry', 'disgust', 'happy', 'sad', 'surprise']

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')  # ¡Solo una vez!

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_resized = cv2.resize(roi_gray, (48, 48))
        roi_pil = Image.fromarray(roi_resized)
        roi_tensor = transform(roi_pil).unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(roi_tensor)
            _, predicted = torch.max(output, 1)
            emotion = classes[predicted.item()]

        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow('Detector de emociones', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
